In [28]:
from experimenter.models import *
from django.db import connection

In [29]:
for experiment in Experiment.objects.all():
    print(experiment)

exp_banner
exp
'0'


In [44]:
with connection.cursor() as curs:
    # check valid experiment
    sql = 'select name from experimenter_experiment;'
    curs.execute(sql)
    rows = curs.fetchall()
    experiment_list = [row[0] for row in rows]

In [45]:
experiment_list

["'0'", 'exp', 'exp_banner']

In [59]:
with connection.cursor() as curs:
    sql = "select g.act_subject from experimenter_goal g, experimenter_experiment e where g.experiment_id=e.id and e.name= '%s';" % ('0')
    curs.execute(sql)
    rows = curs.fetchall()

In [60]:
rows

()

In [ ]:
with connection.cursor() as curs:
    # check valid experiment
    sql = 'select name from experimenter_experiment;'
    curs.execute(sql)
    rows = self.dictfetchall(curs)
    experiment_list = [row['name'] for row in rows]

    if experiment not in experiment_list:
        print('Invalid experiment')
        return None

    # check valid act_subject
    sql = "select g.act_subject from experimenter_goal g, experimenter_experiment e \
                    where g.experiment_id=e.id and e.name= '%s';" % (experiment)
    curs.execute(sql)
    rows = self.dictfetchall(curs)
    act_subject_list = [row['act_subject'] for row in rows]

    if act_subject not in act_subject_list:
        print("Invalid act_subject")
        return None

    # Caculating CTR
    sql = "select * from appserver_rest_useraction where json_extract(groups,'$.%s')='%s' and time < '%s' + INTERVAL 1 DAY;"\
          % (experiment, group, date)
    curs.execute(sql)
    rows = self.dictfetchall(curs)

    # initialize CTR variables
    impressions = 0
    clicks = 0
    # count CTR variables for each rows
    for row in rows:
        if experiment in row['action'] and 'view' in row['action']:
            impressions += 1
        elif act_subject in row['action'] and 'click' in row['action']:
            clicks += 1
    ctr = clicks / impressions

    return round(ctr, 3)